In [1]:
%%capture
!pip install ultralytics
!pip install ensemble-boxes

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import re
import matplotlib.patches as patches
import os
import yaml
import json
import shutil
import torch

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from ensemble_boxes import weighted_boxes_fusion
from ultralytics import YOLO
import wandb
import warnings
warnings.filterwarnings("ignore")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
try:
    api_key = "67c76616e22ff988e55c17cd3e2257b0b71921de"
    wandb.login(key=api_key)
except:
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. '
          'Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: saif-sedaoud. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
class Config:
    # Path
    base_dir = "/kaggle/input/ghana-crop-disease"
    train_imgs_dir = os.path.join(base_dir, "images")
    test_imgs_dir = os.path.join(base_dir, "images")

    # Model
    model_name = 'yolov8m.pt'
    img_res = 1024 # 2048

    # Training
    n_folds = 20
    selected_folds = [0]
    training_params = {
        "epochs": 40,
        "close_mosaic": 10,
        "batch": 12,
        "amp": True,
        "optimizer": "auto",
        "seed": 42,
        "device": [0],
        "verbose": False,
        "resume": False,
        "patience": 20,
        "iou": 0.5,
        "fliplr": 0.2,  # Horizontal flip probability
        "degrees": 0.2,  # Image rotation in degrees
        "translate": 0.1,  # Translation as a fraction of image size
        "scale": 0.2,  # Scale range (e.g., ±50% of the original size)
        "shear": 0.1,  # Shear angle in degrees
        "hsv_h": 0.015,  # Adjust hue (fraction)
        "hsv_s": 0.2,  # Adjust saturation (fraction)
        "hsv_v": 0.1,  # Adjust value (brightness, fraction)
        "flipud": 0.2,  # Vertical flip probability
        "mosaic": 1.0,  # Mosaic augmentation probability
        "erasing": 0.3,  # Random erasing probability
        "crop_fraction": 0.3,  # Crop fraction for cropping-based augmentation
        "cos_lr": True
    }


config = Config()

In [5]:
def get_class_counts(train_df):
    class_counts = {image_id: {i: 0 for i in range(1, config.num_classes+1)} for image_id in train_df["Image_ID"].unique()}
    for image_id in tqdm(train_df["Image_ID"].unique(), desc='Preparing for cross validation'):
        for target in range(1, config.num_classes):
            if len(train_df.loc[(train_df.Image_ID==image_id) & (train_df.DiseaseClass==int(target))]) > 0:
                class_counts[image_id][target] = 1

    return pd.DataFrame(class_counts).T.reset_index().rename(columns={"index": "Image_ID"})

In [6]:
def format_image_id(image_id, ext="png"):
    image_id = str(image_id)
    if "ID_" in image_id and "." in image_id:
        return image_id
    elif "." in image_id:
        return f"ID_{int(image_id):06d}"
    elif "ID_" in image_id:
        return f"{image_id}.{ext}"
    else:
        return f"ID_{int(image_id):06d}.{ext}"

In [7]:
def move_images(dest_dir, df):
    os.makedirs(dest_dir, exist_ok=True)
    unique_imgs = df.Image_ID.unique()
    for img in tqdm(unique_imgs, total=len(unique_imgs), desc="Moving images"):
        shutil.copy(os.path.join(config.train_imgs_dir, img), dest_dir)

In [8]:
from PIL import Image
import numpy as np

def decode_bbox(bbox: list, image_path: str):
    # Extract image dimensions
    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Unpack bounding box coordinates
    x_min, y_min, x_max, y_max = bbox

    # Compute center coordinates, width, and height of the box
    xc = (x_min + x_max) / 2
    yc = (y_min + y_max) / 2
    w = x_max - x_min
    h = y_max - y_min

    # Normalize values by the image dimensions
    xc /= img_width
    yc /= img_height
    w /= img_width
    h /= img_height

    # Combine normalized values into a single string
    box = np.array([xc, yc, w, h])
    return " ".join(f"{i:.4g}" for i in box)


In [9]:
def create_labels(phase, df):
    phase = os.path.join(phase, "labels")
    os.makedirs(phase, exist_ok=True)
    unique_imgs = df.Image_ID.unique()
    for i, img in tqdm(enumerate(unique_imgs), total=len(unique_imgs), desc="Creating labels"):
        df_img = df.loc[df.Image_ID==img]
        boxes = [f'{row.DiseaseClass} {decode_bbox(row[["xmin", "ymin", "xmax", "ymax"]],os.path.join(config.train_imgs_dir,img))}' for _, row in df_img.iterrows()]
        with open(os.path.join(phase,os.path.splitext(img)[0] + ".txt"), "w") as f:
            for box in boxes:
                f.write(box + "\n")

In [10]:
def get_prediction(model, image_id, phase="val"):
    if phase == "val":
        path = config.train_imgs_dir
    else:
        path = config.test_imgs_dir
    pred = model.predict(os.path.join(path,image_id), imgsz=config.img_res, conf=0.3, augment=False, agnostic_nms=True)
    pred = json.loads(pred[0].tojson())
    return pred

In [11]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    # Slicing idx:idx+1 in order to keep tensor dimensionality
    # Doing ... in indexing if there would be additional dimensions
    # Like for Yolo algorithm which would have (N, S, S, 4) in shape
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [12]:
def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=15
):

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)

        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[2:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [13]:
def evaluate_model(model, val_df):
    transformed_pred = []

    for image_id in tqdm(val_df["Image_ID"].unique(), desc="Evaluating"):
        pred = get_prediction(model, image_id)
        for item in pred:
            transformed_pred.append({
                'Image_ID': image_id,
                'DiseaseClass': item['class'],
                'confidence': item['confidence'],
                'Xmin': item['box']['x1'],
                'Ymin': item['box']['y1'],
                'Xmax': item['box']['x2'],
                'Ymax': item['box']['y2']
            })
    transformed_pred_ = [[v for v in pred.values()] for pred in transformed_pred]
    val_df_ = [row.values.tolist() for _, row in val_df.iterrows()]
    mAP = mean_average_precision(transformed_pred_, val_df_, num_classes=config.num_classes)
    transformed_pred = pd.DataFrame(transformed_pred)

    return mAP.item(), transformed_pred

In [14]:
def generate_prediction(model, ss):
    unique_imgs = ss["Image_ID"].unique()
    transformed_pred = []
    for image_id in tqdm(unique_imgs, total=len(unique_imgs), desc="Generaing predictions"):
        pred = get_prediction(model, image_id, phase="test")
        for item in pred:
            transformed_pred.append({
                'Image_ID': image_id,
                'DiseaseClass': int(item['name']),
                'confidence': item['confidence'],
                'Xmin': item['box']['x1'],
                'Ymin': item['box']['y1'],
                'Xmax': item['box']['x2'],
                'Ymax': item['box']['y2']
            })
    transformed_pred = pd.DataFrame(transformed_pred)
    return transformed_pred

In [15]:
train = pd.read_csv(os.path.join(config.base_dir, "Train.csv"))
ss = pd.read_csv(os.path.join(config.base_dir, "SampleSubmission.csv"))

In [16]:
def get_labels(row):
    if row["DiseaseClass"].endswith("Mosaic"):
        return "Mosaic"
    elif row["DiseaseClass"] == "Healthy":
        return row['class']
    else:
        return row["DiseaseClass"]

In [17]:
train['DiseaseClass'] = train['class'].apply(lambda x: '_'.join(x.split('_')[1:]))
train['DiseaseClass'] = train.apply(get_labels, axis=1)
train = train.loc[~train['class'].str.startswith('Corn')].reset_index(drop=True)
train = train.drop('class', axis=1).drop_duplicates(ignore_index=True)
train_df = train.copy(deep=True)

In [18]:
train.Image_ID.nunique()

3493

In [19]:
config.classes = train.DiseaseClass.unique().tolist()
config.num_classes = len(config.classes)
config.id2cls = {k:v for k, v in enumerate(config.classes)}
config.cls2id = {v:k for k, v in enumerate(config.classes)}
print(f"classes: {config.id2cls}")

train["DiseaseClass"] = train["DiseaseClass"].map(config.cls2id)

class_counts = get_class_counts(train)

def get_new_labels(y):
    y_new = LabelEncoder().fit_transform([''.join(str(l)) for l in y.values])
    return y_new

y_new = get_new_labels(class_counts[range(1, config.num_classes+1)])

folds = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=config.training_params["seed"])
class_counts['fold'] = -1
for i, (train_index, test_index) in enumerate(folds.split(class_counts, y=y_new)):
    class_counts.loc[test_index,'fold'] = i

with open("/kaggle/working/data.yaml", "w") as f:
    json.dump({"train": "train", "val": "val", "nc": config.num_classes, "names": config.classes}, f)

cv_scores = []
preds = []
for fold in range(config.n_folds):
    if fold not in config.selected_folds:
        continue
    print(f'--------------------------------Training Fold {fold+1}/5---------------------------------')
    train_ids = class_counts[class_counts.fold!=fold].reset_index(drop=True)
    valid_ids = class_counts[class_counts.fold==fold].reset_index(drop=True)

    train_ = train.loc[train.Image_ID.isin(train_ids.Image_ID.values)]
    val_ = train.loc[train.Image_ID.isin(valid_ids.Image_ID.values)]

    print(f"Training Data: {len(train_)} - unique: {train_.Image_ID.nunique()}")
    print(f"Validating Data: {len(val_)} - unique: {val_.Image_ID.nunique()}")

    train_dir = os.path.join("/kaggle/working/train", "images")
    val_dir = os.path.join("/kaggle/working/val", "images")
    move_images(train_dir, train_)
    move_images(val_dir, val_)

    create_labels("/kaggle/working/train", train_)
    create_labels("/kaggle/working/val", val_)

    model = YOLO(config.model_name)
    model.train(data="/kaggle/working/data.yaml", task="detect", imgsz=config.img_res, val=True,
                
                name=f"Yolo_M_fold_{fold}", **config.training_params)

    model.save("non_corn_yolo_11_nano_40_epoch.pt")
    
    # score, preds_df = evaluate_model(model, val_)
    # cv_scores.append(score)
    # preds.append(generate_prediction(model, ss))

    # Remove directories
    shutil.rmtree("train")
    shutil.rmtree("val")
    # print(f"Score on fold {fold}: {score}")

# print(f"cv score: {np.mean(cv_scores)}")

classes: {0: 'Bacterial_Spot', 1: 'Fusarium', 2: 'Early_Blight', 3: 'Septoria', 4: 'Leaf_Curl', 5: 'Mosaic', 6: 'Pepper_Healthy', 7: 'Tomato_Healthy', 8: 'Late_Blight', 9: 'Cercospora', 10: 'Leaf_Blight'}


Preparing for cross validation: 100%|██████████| 3493/3493 [01:18<00:00, 44.63it/s]


--------------------------------Training Fold 1/5---------------------------------
Training Data: 23710 - unique: 3318
Validating Data: 1215 - unique: 175


Creating labels: 100%|██████████| 175/175 [00:01<00:00, 93.38it/s] 


100%|██████████| 49.7M/49.7M [00:00<00:00, 255MB/s]


...

      40/40      12.5G       1.47      1.037      1.299         29       1024: 100%|██████████| 277/277 [04:43<00:00,  1.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.62it/s]

                   all        175       1215      0.522      0.466      0.461      0.199



40 epochs completed in 3.241 hours.
Optimizer stripped from runs/detect/Yolo_M_fold_0/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/Yolo_M_fold_0/weights/best.pt, 52.1MB

Validating runs/detect/Yolo_M_fold_0/weights/best.pt...
Ultralytics 8.3.72 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.20it/s]


                   all        175       1215      0.506      0.488       0.46      0.201
Speed: 0.4ms preprocess, 21.5ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to runs/detect/Yolo_M_fold_0


In [20]:
## Max MAP50: 0.42 close mosaic=20 / without any augmentations
## Max MAP50: - 0.40 close mosaic=10 / with augmentations mosaic 0.15-0.30 / 10 epochs

In [21]:
# from IPython.display import FileLink

# FileLink('runs/detect/Yolo_M_fold_0/weights/best.pt')